# notebook: run_baseline_pipeline.ipynb

In [2]:
# --- 1. Настройка окружения и импорт ---
import os
import sys
import pandas as pd

# Добавляем путь к проекту, чтобы можно было импортировать из utils
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()
    
BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, ".."))
if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)
    
from utils.data_loader import load_dataset
from utils.config import BASE_DIR, DATA_PATH, EXPERIMENTS_DIR, ENV_PATH
from baseline.core import RelevanceBaseline
from dotenv import load_dotenv
load_dotenv(ENV_PATH) # загружаем .env

# --- 2. Проверка API ключа ---
openai_key = os.getenv("OPENAI_API_KEY")
if openai_key is None:
    raise ValueError("OPENAI_API_KEY не найден в .env файле")

print("OPENAI_API_KEY загружен.")


# --- 3. Загрузка данных ---
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Файл с данными не найден: {DATA_PATH}")

train_data, val_data, test_data = load_dataset(DATA_PATH, drop_uncertain=True, val_frac=0.01)
print(f"Данные загружены. Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

# --- 4. Инициализация модели (бейзлайна) ---
baseline = RelevanceBaseline()

# --- 5. Оценка на валидации ---
print("Запуск на валидации...")
val_preds, val_acc = baseline.run_full_evaluation(val_data, batch_size=5)
print(f"Validation accuracy: {val_acc:.4f}")

# --- 6. Оценка на тесте ---
print("Запуск на тесте...")
test_preds, test_acc = baseline.run_full_evaluation(test_data, batch_size=5)
print(f"Test accuracy: {test_acc:.4f}")

# --- 7. Сохранение результатов ---
os.makedirs(EXPERIMENTS_DIR, exist_ok=True)
val_preds.to_csv(os.path.join(EXPERIMENTS_DIR, "baseline_val_predictions.csv"), index=False)
test_preds.to_csv(os.path.join(EXPERIMENTS_DIR, "baseline_test_predictions.csv"), index=False)
print(f"Результаты сохранены в {EXPERIMENTS_DIR}")  

OPENAI_API_KEY загружен.
Данные загружены. Train: 29592, Val: 299, Test: 500
Запуск на валидации...


Evaluating batches:   0%|          | 0/60 [00:00<?, ?it/s]

Accuracy (по 299 примерам): 0.6856
Validation accuracy: 0.6856
Запуск на тесте...


Evaluating batches:   0%|          | 0/100 [00:00<?, ?it/s]

Accuracy (по 500 примерам): 0.7840
Test accuracy: 0.7840
Результаты сохранены в D:\My_courses\DLS_NLP_24\llm_relevance_agent\experiments


In [3]:
f'Стоимость данного прогона в vsegpt {198.839580 - 149.285580} рублей'

'Стоимость данного прогона в vsegpt 49.554 рублей'